**Comparação SSD e FASTER RCNN**

Este notebook busca explorar os modelos Single Shot Detector e Faster RCNN utilizando a TensorFlow Object Detection API. Os modelos foram treinados com o conjunto de dados COCO dataset.
- Single Shot Detector - SSD: O modelo SSD utiliza uma única rede neural para a detecção de objetos. A imagem é dividida em diversas caixas delimitadoras e dentro destas caixas delimitadoras os objetos são detectados;
- Faster RCNN: Utiliza uma rede de proposta de regiões no lufgar da busca seletiva a fim de utilizar menos recurosos computacionais.

Os modelos foram implementados baseados na API retormencionada e foram realizados testes com dois grupos de imagens. Um grupo de 10 imagens obtidos da internet (Grupo imagens da internet) e um grupo com três imagens presentes no próprio projeto utilizado como base que contém os modelos (Grupo projeto)

**Resultados**

**Grupo imagens da internet**: este grupo é composto por 10 imagens obtidas da internet, ele contém diferentes tipos de objetos. As iamgens possuem dimensões diferentes.
- SSD: O modelo foi executado em 2,4 ns que foi extremamente rápido. O modelo obteve bons resultados, mas não conseguiu distinguir bem todos os objetos. Por exemplo, em uma das imagens que contém um computador, o SSD não conseguiu detectar o mouse.
- Faster RCNN: O modelo foi lento na execução demorando 746 ns para realizar a detecção completa das imagens. Contudo o resultado foi excelente, o modelo conseguiur detectar bem todos os objetos e encontrar objetos pequenos, como, por exemplo, o mouse que o SSD não conseguiu detectar.

**Grupo projeto**: este grupo contém três imagens próprias do projeto base. Neste grupo o SSD se sobresaiu em questão de tempo de execução enquanto a Faster RCNN se sobresaiu na questão de detecção.

Portanto em questões de tempo de execução o modelo YOLO se sobressae enquanto em termos de acurácia na detecção o modelo Faster RCNN se sobresai.

O restante do notebook mostra o desenvolvimento dos experimentos.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Instalando o tensorflow e os pacotes necessários
!pip install tensorflow==2.7.0 # Evitar erros
!pip install pycocotools
!pip install tf_slim

In [ ]:
# Importações necessárias
import os
import pathlib

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

import tensorflow as tf

In [ ]:
# Fazendo o Download da API
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Compilando o modelo
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.

/content/models/research


In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
'''
  Classe base para criar um modelo baseado na API de detecção de objetos do TensorFlow
'''
class ModeloObjectDetectionAPI():
  nome_modelo = ''
  caminho_labels = ''
  indices_categorias = {}
  modelo = None
  output_dict = {}
  nivel_confianca = 0.0

  def __init__(self, nome_modelo = '', caminho_labels = '', nivel_confianca = 0.5):
    self.nome_modelo = nome_modelo
    self.caminho_labels = caminho_labels
    self.indices_categorias = label_map_util.create_category_index_from_labelmap(caminho_labels, use_display_name=True)
    self.nivel_confianca = nivel_confianca
    self.carregar_model()
    
  
  def carregar_model(self):
    '''
      Realiza o download e cria o modelo
    '''
    base_url = 'http://download.tensorflow.org/models/object_detection/'  # Link para baixar o modelo
    model_file = self.nome_modelo + '.tar.gz'

    model_dir = tf.keras.utils.get_file(
      fname=self.nome_modelo, 
      origin=base_url + model_file,
      untar=True)

    model_dir = pathlib.Path(model_dir)/'saved_model'

    model = tf.saved_model.load(str(model_dir))
    model = model.signatures['serving_default']

    self.modelo = model
    
  def executar_deteccao_imagem(self, imagem):
    '''
      Realiza a detecção dos objetos em um única imagem
    '''
    imagem = np.asarray(imagem) # Convertendo para numpy

    input_tensor = tf.convert_to_tensor(imagem) # Convertendo para o formato do tensor
    input_tensor = input_tensor[tf.newaxis, ...] # Adicionando uma nova dimensão no tensor que diz respeito ao batch size

    # Executando a inferência
    self.output_dict = self.modelo(input_tensor)

    # Convertendo para array numpy e removendo a dimensão extra
    num_detections = int(self.output_dict.pop('num_detections'))

    self.output_dict = {key:value[0, :num_detections].numpy() for key, value in self.output_dict.items()}
    self.output_dict['num_detections'] = num_detections

    # Convertendo as classes para int
    self.output_dict['detection_classes'] = self.output_dict['detection_classes'].astype(np.int64)

    # Manipulando os modelos e as máscaras

    if 'detection_masks' in self.output_dict:
      # Corrigindo o tamanho da imagem e adicionado a caixa delimitadora
      detection_reframed = utils_ops.reframe_box_masks_to_image_masks(self.output_dict['detection_masks'],
                                                                      self.output_dict['detection_masks'], self.output_dict['detection_boxes'],
                                                                      imagem.shape[0], imagem.shape[1])
      detection_reframed = tf.cast(detection_reframed > self.nivel_confianca, tf.uint8)
      self.output_dict['detection_masks_reframed'] = detection_reframed.numpy()

    return self.output_dict

  def detectar_mostrar_inferencia(self, imagem_path):
    '''
      Mostra a inferência da imagem
    '''
    # Preparando a imagem para ser dada como entrada para a rede
    imagem_np = np.array(Image.open(imagem_path))
    output_dict = self.executar_deteccao_imagem(imagem_np)

    # Exibindo o resultado
    self.mostrar_inferencia(output_dict, imagem_np)

  def mostrar_inferencia(self, output_dict, imagem_np):
    vis_util.visualize_boxes_and_labels_on_image_array(
    imagem_np,
    output_dict['detection_boxes'],
    output_dict['detection_classes'],
    output_dict['detection_scores'],
    self.indices_categorias,
    instance_masks=output_dict.get('detection_masks_reframed', None),
    use_normalized_coordinates=True,
    line_thickness=8)

    display(Image.fromarray(imagem_np))

**Carregando e processando as imagens**

In [ ]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/drive/MyDrive/Mestrado/Estudos de caso/Dataset-Testes')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))

In [ ]:
TEST_IMAGE_PATHS

[PosixPath('/content/drive/MyDrive/Mestrado/Estudos de caso/Dataset-Testes/baseball.jpg'),
 PosixPath('/content/drive/MyDrive/Mestrado/Estudos de caso/Dataset-Testes/coca-cola.jpg'),
 PosixPath('/content/drive/MyDrive/Mestrado/Estudos de caso/Dataset-Testes/computador2.jpg'),
 PosixPath('/content/drive/MyDrive/Mestrado/Estudos de caso/Dataset-Testes/img-policial.jpg'),
 PosixPath('/content/drive/MyDrive/Mestrado/Estudos de caso/Dataset-Testes/mesa.jpg'),
 PosixPath('/content/drive/MyDrive/Mestrado/Estudos de caso/Dataset-Testes/pessoas.jpg'),
 PosixPath('/content/drive/MyDrive/Mestrado/Estudos de caso/Dataset-Testes/rua01.jpg'),
 PosixPath('/content/drive/MyDrive/Mestrado/Estudos de caso/Dataset-Testes/rua02.jpg'),
 PosixPath('/content/drive/MyDrive/Mestrado/Estudos de caso/Dataset-Testes/volei.jpg')]

In [ ]:
%matplotlib inline

In [ ]:
import cv2
def mostrar_imagem(imagem):
  fig = plt.gcf() # Limpa as configurações do gráfico
  fig.set_size_inches(18, 6)
  plt.imshow(cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB), cmap='gray')
  plt.axis('off')
  plt.show()

In [ ]:
# Exibindo as imagens dos dados de teste
for image in TEST_IMAGE_PATHS:
  mostrar_imagem(cv2.imread(str(image)))

In [ ]:
modelo_fasterRCNN = ModeloObjectDetectionAPI(nome_modelo = 'faster_rcnn_nas_coco_2018_01_28', caminho_labels='/content/models/research/object_detection/data/mscoco_label_map.pbtxt')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
modeloSSD = ModeloObjectDetectionAPI(nome_modelo = 'ssd_mobilenet_v2_coco_2018_03_29', caminho_labels='/content/models/research/object_detection/data/mscoco_label_map.pbtxt')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
import time
from tqdm import tqdm

In [ ]:
def testar_modelos(modelo, test_images):
  dict_deteccoes = []
  image_np = []


  # Faster RCNN
  for image in tqdm(test_images, total = len(test_images)):
    imagem_np = np.array(Image.open(image))
    dict_deteccoes.append(modelo.executar_deteccao_imagem(imagem_np))
    image_np.append(imagem_np)
  
  return dict_deteccoes, image_np

In [ ]:
inicio = time.time()
dict_deteccoes_FasterRCNN, image_np_FasterRCNN = testar_modelos(modelo_fasterRCNN, TEST_IMAGE_PATHS)
fim = time.time()

100%|██████████| 9/9 [12:25<00:00, 82.88s/it]


In [ ]:
tempo_execucao_faster_RCNN = fim - inicio
print(f'Faster RCNN - tempo de execução: {fim - inicio} ns')

Faster RCNN - tempo de execução: 745.9581453800201 ns


In [ ]:
inicio = time.time()
dict_deteccoes_SSD, image_np_SSD = testar_modelos(modeloSSD, TEST_IMAGE_PATHS)
fim = time.time()

100%|██████████| 9/9 [00:02<00:00,  3.77it/s]


In [ ]:
tempo_execucao_faster_RCNN = fim - inicio
print(f'SSD - tempo de execução: {fim - inicio} ns')

SSD - tempo de execução: 2.3994100093841553 ns


In [ ]:
for i in tqdm(range(len(dict_deteccoes_FasterRCNN))):
  nome_img = str(TEST_IMAGE_PATHS[i]).split('/')[-1].split('.')[0]
  print(f'Imagem: {nome_img}')
  modelo_fasterRCNN.mostrar_inferencia(dict_deteccoes_FasterRCNN[i], image_np_FasterRCNN[i])
  modeloSSD.mostrar_inferencia(dict_deteccoes_SSD[i], image_np_SSD[i])

In [ ]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))

In [ ]:
inicio = time.time()
for image_path in TEST_IMAGE_PATHS:
  modeloSSD.detectar_mostrar_inferencia(image_path)
fim = time.time()

In [ ]:
tempo_execucao_faster_RCNN = fim - inicio
print(f'SSD - tempo de execução: {fim - inicio} ns')

SSD - tempo de execução: 5.221132755279541 ns


In [ ]:
inicio = time.time()
for image_path in TEST_IMAGE_PATHS:
  modelo_fasterRCNN.detectar_mostrar_inferencia(image_path)
fim = time.time()

In [ ]:
tempo_execucao_faster_RCNN = fim - inicio
print(f'Faster RCNN - tempo de execução: {fim - inicio} ns')

Faster RCNN - tempo de execução: 256.4962058067322 ns
